## Let's Use Airbnb's API to get Data for Nashville Rentals

In [91]:
import requests
from bs4 import BeautifulSoup  
import pandas as pd

#### Make a request using the requests library

In [175]:
website_url = 'https://www.airbnb.com/s/East-Nashville--Nashville--TN--United-States/homes?refinement_paths%5B%5D=%2Fhomes&tab_id=home_tab&checkin=2020-12-18&checkout=2020-12-20&adults=10&search_type=filter_change&gps_lat=36.1005056&gps_lng=-86.7401728&price_max=510&place_id=ChIJQfjkuXtoZIgRnWBuVONAPJs'

request = requests.get(website_url)

request.status_code

200

#### Use [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) package to extract data from html docs (and xml docs)

In [176]:
bnb = BeautifulSoup(request.content, 'lxml')
print(bnb.title)

<title>Airbnb® | East Nashville, Nashville - Vacation Rentals &amp; Places to Stay - Tennessee, United States</title>


In [177]:
listings = bnb.find_all('div', attrs = {'class': '_8ssblpx'}) # listing level information

In [178]:
type(listings)

bs4.element.ResultSet

In [179]:
len(listings)

20

#### Let's look at what can be scraped from the first page of listings

In [180]:
guest = bnb.find("div", {"class":"_kqh46o"}).text # guests and BR/BA
guest

'12 guests · 4 bedrooms · 11 beds · 3.5 baths'

In [181]:
url = bnb.find("div", {"class":"_1048zci"}).find('a')['href'] # url for the listing
url

'/rooms/40420743?adults=10&check_in=2020-12-18&check_out=2020-12-20&previous_page_section_name=1000&federated_search_id=93309177-8221-42d1-8da8-be23f48ca351'

In [182]:
description = bnb.find("div", {"class":"_1048zci"}).text # full description of rental 
description

'SUPERHOSTEntire house in NashvilleButterfly BNB~Modern Luxury~Giant Rooftop~9Min>DT12 guests · 4 bedrooms · 11 beds · 3.5 bathsWifi · Air conditioning · Kitchen · Free parkingNew lower priceRating 4.92 out of 5;4.9237 reviews\xa0(37)Previous price:$591Discounted price:$356 / nightPrice:$1,212 totalShow details'

In [183]:
superhost = bnb.find("div", {"class":"_1nz9l7j"}).text # Superhost status
superhost

'SUPERHOST'

In [184]:
category = bnb.find("div", {"class":"_1tanv1h"}).text # what type of rental is this (i.e. 'Entire Home in Nashville')
category

'Entire house in Nashville'

In [185]:
name = bnb.find("div", {"class":"_bzh5lkq"}).text #name of Airbnb
name

'Butterfly BNB~Modern Luxury~Giant Rooftop~9Min>DT'

In [186]:
cost_night = bnb.find("div", {"class":"_1fwiw8gv"}).text # cost per night stay
cost_night

'Previous price:$591Discounted price:$356 / night'

In [187]:
cost_total = bnb.find("div", {"class":"_vsjqit"}).text
length_cost = cost_total.find(' total')

cost_total = cost_total[0:length_cost] # total cost for the trip 
cost_total

'Price:$1,212'

In [188]:
#rev = bnb.find("div", {"class":"_vaj62s"}).text # this will break if there are no reviews for a listing 
rev = bnb.find("div", {"class":"_1bbeetd"}).text
print(rev)

#length = rev.find(';')
#stars = rev[0:length] # how many stars out of 5

#length2 = rev.find('(')
#reviews = rev[length2:] # how many reviews 

Rating 4.92 out of 5;4.9237 reviews (37)Previous price:$591Discounted price:$356 / nightPrice:$1,212 totalShow details


#### Let's put all of the listing information for the first page into a dataframe

In [189]:
names, night_cost, categories, suphost, urls, guests, total, reviews = [], [], [], [], [], [], [], []

for num in range(0, 20): 
    names.append(listings[num].find("div", {"class":"_bzh5lkq"}).text) #name of Airbnb
    night_cost.append(listings[num].find("div", {"class":"_1fwiw8gv"}).text) # cost per night stay
    categories.append(listings[num].find("div", {"class":"_1tanv1h"}).text) # what type of rental is this (i.e. 'Entire Home in Nashville')
    suphost.append(listings[num].find("div", {"class":"_1nz9l7j"}).text) # Superhost status
    urls.append(listings[num].find("div", {"class":"_1048zci"}).find('a')['href']) # url for the listing
    guests.append(listings[num].find("div", {"class":"_kqh46o"}).text) # guests and BR/BA
    reviews.append(listings[num].find("div", {"class":"_1bbeetd"}).text) # review count and avg star rating
    total.append(listings[num].find("div", {"class":"_vsjqit"}).text) # total cost for trip 
    
dictionary = {'name': names, 'nightly cost': night_cost, 'category': categories, 'superhost': suphost, 'url': urls, 'guests_rental': guests, 'total cost': total, 'reviews': reviews}
page_data = pd.DataFrame(dictionary)
page_data.head()

,name,nightly cost,category,superhost,url,guests_rental,total cost,reviews
0,Butterfly BNB~Modern Luxury~Giant Rooftop~9Min>DT,Previous price:$591Discounted price:$356 / night,Entire house in Nashville,SUPERHOST,/rooms/40420743?adults=10&check_in=2020-12-18&...,12 guests · 4 bedrooms · 11 beds · 3.5 baths,"Price:$1,212 totalShow details",Rating 4.92 out of 5;4.9237 reviews (37)Previo...
1,Explore Music City from a Sanctuary-like Urban...,Price:$349 / night,Entire house in Nashville,PLUS,/rooms/plus/27520811?adults=10&check_in=2020-1...,10 guests · 4 bedrooms · 6 beds · 3 baths,Price:$968 totalShow details,Rating 5.0 out of 5;5.0112 reviews (112)Price:...
2,4BR from Thomas Rhett Video✦Patio/Deck/Fenced ...,Previous price:$500Discounted price:$365 / night,Entire house in Nashville,SUPERHOST,/rooms/14088893?adults=10&check_in=2020-12-18&...,12 guests · 4 bedrooms · 7 beds · 3 baths,Price:$947 totalShow details,Rating 4.96 out of 5;4.9649 reviews (49)Previo...
3,Newly Built Home Perfect for any Occasion!!!,Previous price:$319Discounted price:$244 / night,Entire house in Nashville,,/rooms/45452826?adults=10&check_in=2020-12-18&...,10 guests · 4 bedrooms · 5 beds · 3 baths,Price:$757 totalShow details,Rating 4.67 out of 5;4.673 reviews (3)Previous...
4,"East Nashville Home, walk to dining & greenway",Price:$275 / night,Entire house in Nashville,,/rooms/45609934?adults=10&check_in=2020-12-18&...,12 guests · 5 bedrooms · 8 beds · 3 baths,Price:$787 totalShow details,Price:$275 / nightPrice:$787 totalShow details


#### Iterate through the rest of the pages and pull the listing data

#### Create a list of all of the pages for the search criteria 

In [190]:
urls = []
names, night_cost, categories, suphost, list_urls, guests, total, reviews = [], [], [], [], [], [], [], []

for i in range(1, 15):
    url = website_url + '&items_offset=' + str(i * 20) + '&section_offset=3'
    urls.append(url)
    i = i+1

for j in range(0, 14):
    page = urls[j]
    response2 = requests.get(page)
    bnb2 = BeautifulSoup(response2.content, 'lxml')
    listings2 = bnb2.find_all('div', attrs = {'class': '_8ssblpx'})
    
    for num2 in range(0, 20): 
        names.append(listings2[num2].find("div", {"class":"_bzh5lkq"}).text) #name of Airbnb
        night_cost.append(listings2[num2].find("div", {"class":"_1fwiw8gv"}).text) # cost per night stay
        categories.append(listings2[num2].find("div", {"class":"_1tanv1h"}).text) # what type of rental is this (i.e. 'Entire Home in Nashville')
        suphost.append(listings2[num2].find("div", {"class":"_1nz9l7j"}).text) # Superhost status
        list_urls.append(listings2[num2].find("div", {"class":"_1048zci"}).find('a')['href']) # url for the listing
        guests.append(listings2[num2].find("div", {"class":"_kqh46o"}).text) # guests and BR/BA
        reviews.append(listings2[num2].find("div", {"class":"_1bbeetd"}).text) # review count and avg star rating
        total.append(listings2[num2].find("div", {"class":"_vsjqit"}).text) # total cost for trip 

dictionary = {'name': names, 'nightly cost': night_cost, 'category': categories, 'superhost': suphost, 'url': list_urls, 'guests_rental': guests, 'total cost': total, 'reviews': reviews}
search_data = pd.DataFrame(dictionary)

search_data

,name,nightly cost,category,superhost,url,guests_rental,total cost,reviews
0,BRAND NEW HOME ★ HUGE ROOFTOP ★ LARGE 4BR,Price:$399 / night,Entire house in Nashville,SUPERHOST,/rooms/23825123?adults=10&check_in=2020-12-18&...,12 guests · 4 bedrooms · 7 beds · 2.5 baths,"Price:$1,082 totalShow details",Rating 4.74 out of 5;4.7473 reviews (73)Price:...
1,"SUPER CLEAN LUXURY HOME, ONSITE BBQ AND TAILGA...",Previous price:$263Discounted price:$185 / night,Entire townhouse in Nashville,SUPERHOST,/rooms/40539365?adults=10&check_in=2020-12-18&...,10 guests · 3 bedrooms · 6 beds · 3 baths,Price:$593 totalShow details,Rating 4.89 out of 5;4.8927 reviews (27)Previo...
2,Spacious Modern Resort in East Nashville,Price:$175 / night,Entire townhouse in Nashville,,/rooms/45200659?adults=10&check_in=2020-12-18&...,14 guests · 4 bedrooms · 5 beds · 3.5 baths,Price:$794 totalShow details,Rating 4.67 out of 5;4.673 reviews (3)Price:$1...
3,GULCH GETAWAY! STUNNING VIEWS OF NASHVILLE! 20...,Previous price:$481Discounted price:$290 / night,Entire townhouse in Nashville,,/rooms/31090078?adults=10&check_in=2020-12-18&...,10 guests · 3 bedrooms · 6 beds · 3.5 baths,Price:$889 totalShow details,Rating 4.89 out of 5;4.8953 reviews (53)Previo...
4,HT2 - Beautiful New Home in Five Points with 2...,Previous price:$536Discounted price:$430 / night,Entire house in Nashville,,/rooms/45224489?adults=10&check_in=2020-12-18&...,12 guests · 5 bedrooms · 9 beds · 4 baths,"Price:$1,381 totalShow details",Previous price:$536Discounted price:$430 / nig...
...,...,...,...,...,...,...,...,...
275,Unwind On The SANITIZED Rooftop in Luxury,Previous price:$488Discounted price:$426 / night,Entire house in Nashville,SUPERHOST,/rooms/16080524?adults=10&check_in=2020-12-18&...,10 guests · 3 bedrooms · 6 beds · 4 baths,"Price:$1,199 totalShow details",Rating 4.92 out of 5;4.9263 reviews (63)Previo...
276,Brand New Build / 2 Miles to Downtown,Price:$490 / night,Entire house in Nashville,SUPERHOST,/rooms/38693715?adults=10&check_in=2020-12-18&...,10 guests · 3 bedrooms · 5 beds · 4 baths,"Price:$1,341 totalShow details",Rating 4.93 out of 5;4.9330 reviews (30)Price:...
277,Group listing for 2 units! 1 Mile from Broadway!,Price:$326 / night,Entire apartment in Nashville,SUPERHOST,/rooms/42973154?adults=10&check_in=2020-12-18&...,12 guests · 4 bedrooms · 6 beds · 2 baths,Price:$972 totalShow details,Price:$326 / nightPrice:$972 totalShow details
278,Nashville Modern Luxury - 5 minutes to Downtown!,Price:$450 / night,Entire house in Nashville,,/rooms/18635166?adults=10&check_in=2020-12-18&...,10 guests · 3 bedrooms · 5 beds · 2 baths,"Price:$1,141 totalShow details",Rating 5.0 out of 5;5.08 reviews (8)Price:$450...


In [191]:
first_search = page_data.append(search_data)
first_search.shape

(300, 8)